# October 16

**Conflict**

![](https://media.giphy.com/media/3oz8xABuMZ7AsrcMc8/giphy.gif)

# Warm Up 
It is possible to have multiple owners of a GitHub repository. For our warm-up today, we're going to do an exercise. Get into pairs.

1. Partner 1 & 2: Navigate to the github page for your Lastname_project
2. Partner 1: Go to "Settings." Click "Collaborators." Enter your partner's GitHub name.
3. Partner 2: You might get an email notification saying that you've been added.
4. Partner 2: Clone your partner's repo. Copy the repository location by clicking on the big, green "Clone or download" button. Open a new terminal, and type:

```git clone ``` followed by the address you copied.
5. Partner 2: Now, pick a file. Change something in it. Doesn't matter what, but it has to be something were there is already text on that line.
6. Partner 2: Add, commit and push your code.
7. Partner 1: Pull in your partner's changes. Do you see conflict?
8. Partner 1: Open the conflict file in a text editor, and let me know once you have your conflict.

# Resolving Conflict

This is a little bit social, and a little bit technical. You made a change to something that was. And in a co-owned repo, no one needs to approve your changes. You just get to make them! 

So how can we resolve these issues? 

1. **Read** First, I normally look at the changes. What did this person do? Can I see why? 
2. **Ask** If I don't understand, I ask for help.
3. **Decide** Then I do the merge, and add and commit the code. 


# GBIF

We're at a bit of a crossroads in the course: a lot of the end part of the class is about making websites, and other forms of communicating results. But I think it would help if we did a little more fun programming stuff first.

We're going to get some occurrence data off of GBIF, the website for the Global Biodiversity Information Facility website. We're going to see if we can get some information on where these salamanders live.

First, we will get the names of the salamanders. I've put an empty cell below, and below it an answer. See if you can read in the `plethodon.phy` file with Dendropy, and get the names of our salamanders.


In [ ]:
## Answer below
import dendropy

sal_dat = dendropy.DnaCharacterMatrix.get_from_path("../data/plethodon.phy", schema="phylip")
sal_names = sal_dat.taxon_namespace.labels()

Next, we'll use a library called `pygbif`, which interfaces with GBIF to get locality information for our salamanders. We will search each salamander name against the GBIF database to get locations where that salamander is found. We are only going to do this for a couple salamanders, to keep the exercise tractable for all of us to do.

In [ ]:
import pandas as pd
from pygbif import occurrences

list_of_dictionaries = []

for name in sal_names[1:5]:
    sal_dict = occurrences.search(scientificName = name)
    list_of_dictionaries.append(sal_dict)


There are 53 records for the salamaders in our sample - all our salamanders were found.  The data download as a dictionary, and for ease of processing, we will change these into dataframes. If you view one of the dictionaries, there's a lot of padding - we really only want the "results" entry.

In [ ]:
import pandas as pd

dict_one = list_of_dictionaries[0]['results']
sal_df = pd.DataFrame.from_dict(dict_one)

for item in list_of_dictionaries:
    temp_df = pd.DataFrame.from_dict(item['results'])
    sal_df = sal_df.append(temp_df)

If you look at the dataframe, you'll see that there's a lot of information here, and lots of different ways that we could subset the data. For example, perhaps we only want records where a person physically saw the salamander: 

In [ ]:
sal_slim = sal_df[sal_df.basisOfRecord == "HUMAN_OBSERVATION"]

This is still quite a bit of data! However, many databases don't process null records, or give an error if the record is null. So, let's remove the NaN values, and also save our work to a file. That way if the internet dies, or something, we still have our searches. 

In [ ]:
import numpy as np

locs = sal_slim.dropna(subset=['verbatimLocality'])
locs.to_csv("../data/locs.csv")

What do you notice about these localities? 

We'll need to locate them to coordinates before we can plot them. To do this, we will use an open-source package call geopy, which takes in a string of a location and searches that string against a global map to get Lat and Long coordinates. We will use the values in the `locs` `verbatimLocality` column to do this. 

## Exercise Two: Talk the below code out with a partner. Decide what it does, and then run it to confirm.

In [ ]:
from geopy.geocoders import Nominatim
import numpy as np

geolocator = Nominatim(user_agent="class")
locs['coords'] = 0
locations = []
for row in locs.verbatimLocality.iteritems():
    latlong = geolocator.geocode(row[1])
    if latlong is None:
        locations.append(0)
    else: 
        locations.append(latlong[1])


We now have a list. Use the below cell to confirm that the list is the same length as the locations dataframe.

This is an important step called `testing`. What would it mean if the list was not the same length? Would we want to keep using it? 

Next we will append the list as a new column. 

In [ ]:
se = pd.Series(locations)
locs['coords'] = se.values

We will now do three last activities: 
- Write this raw data into our data folder as a CSV file.
- Decide how we want to treat missing values (0-values), and apply this treatment.
- Write out the treated data as separate from the raw data. 